In [15]:
%reload_ext autoreload
%autoreload 2

import cv2

import os
import sys
sys.path.append(os.path.join(os.environ['REPO_DIR'], 'utilities'))
from utilities2015 import *

from collections import defaultdict

from matplotlib.path import Path
%matplotlib inline

import pandas as pd

from joblib import Parallel, delayed

from scipy.interpolate import RectBivariateSpline
from skimage.transform import rescale, resize

In [16]:
labels =  ['BackG', '5N', '7n', '7N', '12N', 'Gr', 'LVe', 'Pn', 'SuVe', 'VLL']
label_dict = dict([(l,i) for i, l in enumerate(labels)])

colors = (np.loadtxt(os.environ['REPO_DIR'] + '/visualization/100colors.txt')*255).astype(np.uint8)
colors[label_dict['BackG']] = 255

In [25]:
# stack = 'MD589'
stack = 'MD594'
dm = DataManager(stack=stack, load_mask=False)

In [26]:
patch_dir = '/oasis/projects/nsf/csd395/yuncong/CSHL_data_patches/'
table_filepath = os.path.join(patch_dir, '%(stack)s_indices_allROIs_allSections.h5'%{'stack':stack})
indices_allROIs_allSections = pd.read_hdf(table_filepath, 'indices_allROIs_allSections')
grid_parameters = pd.read_hdf(table_filepath, 'grid_parameters')

In [27]:
patch_size, stride, w, h = grid_parameters.tolist()

print 'patch_size', patch_size
print 'stride', stride
print 'w, h', w, h

half_size = patch_size/2
ys, xs = np.meshgrid(np.arange(half_size, h-half_size, stride), np.arange(half_size, w-half_size, stride),
                     indexing='xy')
sample_locations = np.c_[xs.flat, ys.flat]

patch_size 224
stride 56
w, h 14432 11584


In [ ]:
indices_allROIs_allSections

In [28]:
prectionMap_dir = os.path.join(patch_dir, stack+'_roi1_predictionMaps')

In [29]:
vizOverlaid_output_dir = os.path.join(patch_dir, stack+'_densePredictionMapOverlaid')

if not os.path.exists(vizOverlaid_output_dir):
    os.makedirs(vizOverlaid_output_dir)

In [30]:
vizScoreOverlaid_output_dir = os.path.join(patch_dir, stack+'_denseScoreMapOverlaid')

if not os.path.exists(vizScoreOverlaid_output_dir):
    os.makedirs(vizScoreOverlaid_output_dir)

In [31]:
densePrectionMapLossless_dir = os.path.join(patch_dir, stack+'_roi1_densePredictionMapsLossless')
if not os.path.exists(densePrectionMapLossless_dir):
    os.makedirs(densePrectionMapLossless_dir)
    
denseScoreMapLossless_dir = os.path.join(patch_dir, stack+'_roi1_denseScoreMapsLossless')
if not os.path.exists(denseScoreMapLossless_dir):
    os.makedirs(denseScoreMapLossless_dir)

In [ ]:
# vizOverlaid_output_dir = os.path.join(patch_dir, stack+'_densePredictionMapOverlaid_spm')
# if not os.path.exists(vizOverlaid_output_dir):
#     os.makedirs(vizOverlaid_output_dir)

# densePrectionMapLossless_dir = os.path.join(patch_dir, stack+'_roi1_densePredictionMapsLossless_spm')
# if not os.path.exists(densePrectionMapLossless_dir):
#     os.makedirs(densePrectionMapLossless_dir)

In [32]:
indices_allROIs_allSections

,157,158,159,160,161,162,163,164,165,166
roi1,"[11244, 11245, 11246, 11247, 11248, 11249, 112...","[11244, 11245, 11246, 11247, 11248, 11249, 112...","[11244, 11245, 11246, 11247, 11248, 11249, 112...","[11244, 11245, 11246, 11247, 11248, 11249, 112...","[11244, 11245, 11246, 11247, 11248, 11249, 112...","[11244, 11245, 11246, 11247, 11248, 11249, 112...","[11244, 11245, 11246, 11247, 11248, 11249, 112...","[11244, 11245, 11246, 11247, 11248, 11249, 112...","[11244, 11245, 11246, 11247, 11248, 11249, 112...","[11244, 11245, 11246, 11247, 11248, 11249, 112..."


In [33]:
indices_allROIs_allSections.keys()

Int64Index([157, 158, 159, 160, 161, 162, 163, 164, 165, 166], dtype='int64')

In [ ]:
for sec in indices_allROIs_allSections.keys():
# for sec in [161]:
    
    print sec

    dataset = '%(stack)s_%(sec)04d_roi1' % {'stack': stack, 'sec': sec}
#     cv2.imwrite(os.path.join(patch_dir, dataset + '.jpg'), viz[..., ::-1])
    
    indices_roi = indices_allROIs_allSections[sec]['roi1']
    sample_locations_roi = sample_locations[indices_roi]
    
    interpolation_xmin, interpolation_ymin = sample_locations_roi.min(axis=0)
    interpolation_xmax, interpolation_ymax = sample_locations_roi.max(axis=0)
    interpolation_w = interpolation_xmax - interpolation_xmin + 1
    interpolation_h = interpolation_ymax - interpolation_ymin + 1
    
    print sample_locations_roi.shape[0], 'samples'
    
    probs = np.load(os.path.join(prectionMap_dir, '%(dataset)s_test_prediction_map.npy' % {'dataset': dataset}))
    preds = np.argmax(probs, axis=1).astype(np.int)
    
#     with open('/oasis/projects/nsf/csd395/ruogu/svm4/result/589/0161.txt', 'r') as f:
#         preds = np.array([int(pred) for fn, pred in sorted([l.split() for l in f.readlines()])])
#     probs = np.zeros((len(preds), len(labels)))
#     probs[range(len(preds)), preds] = 1
    
    ##### sample_locations_roi + scores to dense_score_map #####
    
    shrink_factor = 4
    
    sample_locations_unique_xs = np.unique(sample_locations_roi[:,0])
    sample_locations_unique_ys = np.unique(sample_locations_roi[:,1])
    
    n_sample_x = sample_locations_unique_xs.size
    n_sample_y = sample_locations_unique_ys.size
    
    index_x = dict([(j,i) for i,j in enumerate(sample_locations_unique_xs)])
    index_y = dict([(j,i) for i,j in enumerate(sample_locations_unique_ys)])
    sample_location_indices = np.asarray([(index_x[x], index_y[y]) for x, y in sample_locations_roi])
    
    sample_locations_interpolatedArea_ys_matrix, \
    sample_locations_interpolatedArea_xs_matrix = np.meshgrid(range(interpolation_ymin/shrink_factor, 
                                                                    interpolation_ymax/shrink_factor), 
                                                              range(interpolation_xmin/shrink_factor, 
                                                                    interpolation_xmax/shrink_factor), 
                                                              indexing='ij')

    def generate_score_map(l):

        score_matrix = np.zeros((n_sample_x, n_sample_y))
        score_matrix[sample_location_indices[:,0], sample_location_indices[:,1]] = probs[:,l]

        spline = RectBivariateSpline(sample_locations_unique_xs/shrink_factor, 
                                     sample_locations_unique_ys/shrink_factor, 
                                     score_matrix, 
                                     bbox=[interpolation_xmin/shrink_factor, 
                                           interpolation_xmax/shrink_factor, 
                                           interpolation_ymin/shrink_factor, 
                                           interpolation_ymax/shrink_factor])

        dense_score_map = spline.ev(sample_locations_interpolatedArea_xs_matrix, sample_locations_interpolatedArea_ys_matrix)
        
        dense_score_map = resize(dense_score_map, (interpolation_h, interpolation_w))

        return dense_score_map


    import time
    t = time.time()

    dense_score_maps = Parallel(n_jobs=12)(delayed(generate_score_map)(l) for l in range(10))

    print time.time() - t
    
#     dense_sorted_score_map = np.sort(dense_score_maps, axis=0)
#     dense_top_score_map = dense_sorted_score_map[-1]
#     dense_second_score_map = dense_sorted_score_map[-2]

#     dense_prediction_map = np.argsort(dense_score_maps, axis=0)
#     dense_top_prediction_map = dense_prediction_map[-1]
    
    margin = .2
#     margin = .01
    
#     dense_top_prediction_map[dense_top_score_map - dense_second_score_map < margin] = label_dict['BackG']

#     dense_top_prediction_map_lossless = np.pad(dense_top_prediction_map, 
#                                ((interpolation_ymin, h-interpolation_ymax-1), (interpolation_xmin, w-interpolation_xmax-1)),
#                                        mode='constant', constant_values=label_dict['BackG'])
    
#     np.savez_compressed(os.path.join(densePrectionMapLossless_dir, 
#                                      dataset+'_denseTopPredictionMapLossless_margin%.2f'%margin), 
#                         dense_top_prediction_map_lossless)
    

#     dense_score_map_lossless = np.pad(dense_sorted_score_map, 
#                                       ((0,0),
#                                        (interpolation_ymin, h-interpolation_ymax-1), 
#                                        (interpolation_xmin, w-interpolation_xmax-1)),
#                                       mode='constant', constant_values=0)

#     np.savez_compressed(os.path.join(denseScoreMapLossless_dir, dataset+'_denseScoreMapLossless'), 
#                     dense_score_map_lossless)

    
    for i, m in enumerate(dense_score_maps):
        dense_score_map_lossless = np.pad(m, ((interpolation_ymin, h-interpolation_ymax-1), 
                                              (interpolation_xmin, w-interpolation_xmax-1)),
                                          mode='constant', constant_values=0)
        
        np.savez_compressed(os.path.join(denseScoreMapLossless_dir, dataset+'_denseScoreMapLossless_%d'%i), 
                            dense_score_map_lossless)


In [ ]:
label_polygons = pd.read_hdf('/home/yuncong/Brain/database/'+stack+'_label_polygons.h5', 'label_polygons')

In [ ]:
margin = .2

# for sec in indices_allROIs_allSections.keys():
for sec in [163]:
    
    print sec
    
    shrink_factor = 4
    
    dataset = '%(stack)s_%(sec)04d_roi1' % {'stack': stack, 'sec': sec}

    dense_top_prediction_map = np.load(os.path.join(densePrectionMapLossless_dir, 
                                    dataset+'_denseTopPredictionMapLossless_margin%.2f.npz'%margin))['arr_0']
    
    dense_top_prediction_map_downsampled = dense_top_prediction_map[::shrink_factor, ::shrink_factor]
    dense_prediction_map_overlay = colors[dense_top_prediction_map_downsampled].astype(np.uint8)
    
#     cv2.imwrite(os.path.join(patch_dir, dataset + '_densePredictionMap_margin%.2f.jpg'%margin), 
#                 dense_prediction_map_overlay[..., 3::-1])
    
    dm.set_slice(sec)
    dm._load_image(['rgb-jpg'], force_reload=True)
    
    viz = dm.image_rgb_jpg[::shrink_factor, ::shrink_factor].copy()
    
    if sec in label_polygons.index:    
        available_labels = list(label_polygons.loc[sec].dropna().index)
        for l in available_labels:
            if l in label_dict.keys():
                vs = label_polygons[l][sec].astype(np.int) / shrink_factor
                cv2.polylines(viz, [vs], True, colors[label_dict[l]].astype(np.int), 4)    
    
    viz_overlaid = img_as_ubyte(alpha_blending(dense_prediction_map_overlay, viz, .2, 1.))
    
    cv2.imwrite(os.path.join(patch_dir, vizOverlaid_output_dir, 
                             dataset + '_densePredictionMapOverlaid_margin%.2f.jpg'%margin), 
                viz_overlaid[..., [2,1,0,3]])
        

In [15]:
display(dense_score_map_overlay)

/oasis/projects/nsf/csd181/yuncong/virtualenv-1.9.1/yuncongve/lib/python2.7/site-packages/skimage/io/_io.py:159: UserWarning: tmp.jpg is a low contrast image
  warnings.warn('%s is a low contrast image' % fname)


/oasis/projects/nsf/csd395/yuncong/Brain/learning/tmp.jpg

In [14]:
margin = .2

# for sec in indices_allROIs_allSections.keys():
for sec in [163]:
    
    print sec
    
    shrink_factor = 4
    
    dataset = '%(stack)s_%(sec)04d_roi1' % {'stack': stack, 'sec': sec}

    dense_score_map = np.load(os.path.join(denseScoreMapLossless_dir, 
                                    dataset+'_denseScoreMapLossless_2.npz'))['arr_0']
    
    dense_score_map_downsampled = dense_score_map[::shrink_factor, ::shrink_factor]
    dense_score_map_overlay = img_as_ubyte(plt.cm.coolwarm(dense_score_map_downsampled)[..., :3]).astype(np.uint8)
    
    cv2.imwrite(os.path.join(patch_dir, vizScoreOverlaid_output_dir, dataset + '_denseScoreMap_2.jpg'), 
                dense_score_map_overlay[..., 3::-1])
    
#     dm.set_slice(sec)
#     dm._load_image(['rgb-jpg'], force_reload=True)
    
#     viz = dm.image_rgb_jpg[::shrink_factor, ::shrink_factor].copy()
    
# #     if sec in label_polygons.index:    
# #         available_labels = list(label_polygons.loc[sec].dropna().index)
# #         for l in available_labels:
# #             if l in label_dict.keys():
# #                 vs = label_polygons[l][sec].astype(np.int) / shrink_factor
# #                 cv2.polylines(viz, [vs], True, colors[label_dict[l]].astype(np.int), 4)    
    
#     viz_overlaid = img_as_ubyte(alpha_blending(dense_score_map_overlay, viz, .2, 1.))
    
#     cv2.imwrite(os.path.join(patch_dir, vizScoreOverlaid_output_dir, 
#                              dataset + '_denseScoreMapOverlaid_1.jpg'), 
#                 viz_overlaid[..., [2,1,0,3]])
        

163


In [ ]:
plt.matshow(dense_top_prediction_map_lossless[::8, ::8])

In [ ]:
dense_score_map = dense_score_maps[label_dict['5N']]

dense_score_map[dense_score_map > 1] = 1
dense_score_map[dense_score_map < 0.5] = 0

In [ ]:
prob_map_viz = img_as_ubyte(plt.cm.magma(dense_score_map)[...,:3])

In [ ]:
viz2 = alpha_blending(prob_map_viz[...,:3], img_as_ubyte(viz), .2, 1.,)
plt.figure(figsize=(20,20));
plt.imshow(viz2);